In [1]:
from __init__ import PRP; import sys
sys.path.append(PRP)

In [3]:
from scripts.grad_compare import *

In [10]:
acc = warmup_acc(20, override_settings={'enable_streamfunction' : False})
acc.state._diagnostics = {}
current_state = acc.state

Running model setup
 Running diagnostic "averages" every 5.0 days
 Writing output for diagnostic "averages" every 1.0 years
 Running diagnostic "energy" every 5.0 days
 Writing output for diagnostic "energy" every 7.6 days
 Running diagnostic "overturning" every 5.0 days
 Writing output for diagnostic "overturning" every 7.6 days
 Writing output for diagnostic "snapshot" every 10.0 days
 Writing output for diagnostic "tracer_monitor" every 30.4 days
Diffusion grid factor delta_iso1 = 0.01942284820457075


 80%|████████  | 16/20 [00:04<00:00,  5.67it/s]

 Writing snapshot at 10.00 days


100%|██████████| 20/20 [00:05<00:00,  3.80it/s]


In [14]:
def step_function_(function, state) :
    """
    Perform a single step of the simulation in a pure way (no inplace modification)

    Args:
        function: The step function to apply.
        state: The current state of the simulation.

    Returns:
        The next state of the simulation.
    """

    next_state = state.copy()
    function(next_state)
    return next_state

# Params auto-diff
def agg_sum(state, key_sum = 'u', cv = slice(-5,-1,1)) :
    return ((getattr(state.variables, key_sum)[:,:,:] - 0)**2).mean()


diff_function = lambda state : agg_sum(step_function_(acc.step, state))

In [ ]:
l, vjp_fun  = jax.vjp(diff_function, current_state)


# Backward pass using VJP
cotangent = jnp.ones_like(l)

# Backpropagate through self.agg_function
ds, = vjp_agg(cotangent)

In [ ]:

methods = [numerical_diff, jvp_grad, vjp_grad]#[numerical_diff, backward_diff, forward_diff, jvp_grad, vjp_grad]

for ad_method in methods :
    ad = ad_method(step_function, agg_function,  var_name)

    start_time = time.time()
    n_diff = ad.g(acc.state, var_value = var, iterations=iteration_grad)
    print(f'{ad} : ', n_diff)
    n_diff_time = time.time() - start_time
    print(f'Time taken for {ad}: {n_diff_time:.4f} seconds - its={iteration_grad} \n')


In [5]:
step_function = acc.step
agg_function =agg_sum
var_name = 'r_bot'
var = jnp.array(1e-5, dtype=jnp.float64)
iteration_grad = 5

NameError: name 'acc' is not defined